<a href="https://www.kaggle.com/code/martiritter/housing-prices-description-preprocessing?scriptVersionId=262104204" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Preprocessing Housing Prices Data Description

Importing regex for text processing and json because we will export to JSON.

In [1]:
import re
import json

header_regex = r"(?:^|\n)(?P<name>\w+): (?P<description>[\S ]+)"
value_regex = r"\s+(?P<value>[\S ]+?)\s*\t(?P<meaning>[\S ]+)"

txt_string = open(r"/kaggle/input/home-data-for-ml-course/data_description.txt").read()

Our output will be a JSON mapping the column names to their description, potential discrete values, and the meaning of these values.
Additional fields are:
- factor: whether the columns contains factors (discrete, potentially non-ordinal values), derived from whether there are discrete values
- na_is_factor: whether NA counts as a factor... will be relevant for preprocessing
- ordered: if this is a factor, whether it is ordinal, so whether there is a order to the factor values (we just take them as they're listed, with one exception mentioned below)
- values_to_drop: An empty list that contains values to be replaced with NA during preprocessing, but after na_is_factor is applied... used for one special case, to avoid issues with the order of factors

In [2]:
headers = re.findall(header_regex, txt_string)
value_txt_chunks = re.split(header_regex, txt_string)[::3][1:]
values = [re.findall(value_regex, value_string) for value_string in value_txt_chunks]

columns, descriptions = [h[0] for h in headers], [h[1] for h in headers]
discrete_values_list = [[v[0] for v in vl] if vl else [] for vl in values]
value_meanings_list = [[v[1] for v in vl] if vl else [] for vl in values]

description_dict = {
    col: {
        "description": description, 
        "discrete_values": discrete_values, 
        "value_meanings": value_meanings,
        "factor": True if len(discrete_values) > 0 else False,
        "na_is_factor": True if "NA" in discrete_values else False,
        "ordered": False,
        "values_to_drop": []
        } for col, description, discrete_values, value_meanings 
        in zip(columns, descriptions, discrete_values_list, value_meanings_list)
}

We define a subset of columns to be ordered. This is a semi-arbitrary list based on my understanding of the discrete values contained in these columns and can be adapted for personal preference.

In [3]:
ordered_cols = [
    "Street",
    "Alley",
    "LotShape",
    "Utilities",
    "LandContour",
    "LandSlope",
    "OverallQual",
    "OverallCond",
    "ExterQual",
    "ExterCond",
    "BsmtQual",
    "BsmtCond",
    "BsmtExposure",
    "BsmtFinType1",
    "BsmtFinType2",
    "HeatingQC",
    "CentralAir",
    "Electrical",
    "KitchenQual",
    "Functional",
    "FireplaceQu",
    "GarageFinish",
    "GarageQual",
    "GarageCond",
    "PavedDrive",
    "PoolQC",
    ]

for ordered_col in ordered_cols:
    description_dict[ordered_col]["ordered"] = True

Three minor modifications are necessary:
- Alley: For some reason this is ordered ("Gravel", "Paved", "Not existing")... obviously a gravel alley is better than no alley access. The discrete values order is modified to follow this logic, based on the order present in PavedDrive (Paved > Partial Paved > Gravel).
- Electrical: We add "Mix" to the values to be replaced with nan and remove it from the discrete values, as it otherwise does not fit in with the order of the other discrete values. This order was derived from the given description of the entries ("Average", "Fair", "Poor") and follows (SBrkr > FuseA > FuseF > FuseP).

In [4]:
print(description_dict["Alley"]["discrete_values"])
target_order = ["Pave", "Grvl", "NA"]
target_indices = [description_dict["Alley"]["discrete_values"].index(val) for val in target_order]
description_dict["Alley"]["value_meanings"] = [description_dict["Alley"]["value_meanings"][i] for i in target_indices]
description_dict["Alley"]["discrete_values"] = [description_dict["Alley"]["discrete_values"][i] for i in target_indices]
print(description_dict["Alley"]["discrete_values"])

print(description_dict["Electrical"]["discrete_values"])  # Semi-arbitrary, but Mix should not be in the same scale as the other factors
description_dict["Electrical"]["discrete_values"].pop(-1)
description_dict["Electrical"]["values_to_drop"] = ["Mix"]
print(description_dict["Electrical"]["discrete_values"])

['Grvl', 'Pave', 'NA']
['Pave', 'Grvl', 'NA']
['SBrkr', 'FuseA', 'FuseF', 'FuseP', 'Mix']
['SBrkr', 'FuseA', 'FuseF', 'FuseP']


We export the finished preprocessing dict to JSON, so that others can also follow what we do when we automate this step.

In [5]:
json.dump(description_dict, open(r"/kaggle/working/HousingPricesPreprocessing.json", "w"), indent=2)